# Processing of large datasets (near GPU Memory size) with cuDF pandas Accelerator Mode  
<a href="https://github.com/rapidsai/cudf">cuDF</a> is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API in the style of pandas.

cuDF now provides a <a href="https://rapids.ai/cudf-pandas/">pandas accelerator mode</a> (`cudf.pandas`), allowing you to bring accelerated computing to your pandas workflows without requiring any code change.

This notebook demonstrates how the memory management automation added to `cudf.pandas`accelerates processing of much larger datasets. Now, `cudf.pandas` uses a managed memory pool by default which allows it to process datasets larger than the memory of the GPU it is running on. Managed memory prefetching is also enabled by default to improve memory access performance. For more information on CUDA Unified Memory (managed memory), performance, and prefetching, see this <a href="https://developer.nvidia.com/blog/improving-gpu-memory-oversubscription-performance/">NVIDIA Developer blog post</a>

# ⚠️ Verify your setup

First, we'll verify that you are running with an NVIDIA GPU.

In [1]:
!nvidia-smi

Thu Aug  8 04:25:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Download the data

The data we will be working with lists approximately 90 million transactions with relatively higher illicit (HI) activity.

We're downloading a curated copy of Kaggle dataset titled <a ref="https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml?select=HI-Large_Trans.csv"> IBM Anti-Money Laundering Dataset </a> from a GCP bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about a minute.

**Data License and Terms** <br>
As this dataset originates from a Kaggle dataset, it's governed by that dataset's license and terms of use, which is the `Community Data License Agreement – Sharing, Version 1.0`, review here: https://cdla.dev/sharing-1-0/.

**Are there restrictions on how I can use this data? </br>**
For each dataset the user elects to use, the user is responsible for checking if the dataset license is fit for the intended purpose.

In [2]:
%%time
# from the bucket using gsutil command line tool-
!gsutil cp gs://rapidsai/pno-oom-demo/HI-Large_Trans_reduced.parquet /content/

Copying gs://rapidsai/pno-oom-demo/HI-Large_Trans_reduced.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][  2.5 GiB/  2.5 GiB]   50.3 MiB/s                                   
Operation completed over 1 objects/2.5 GiB.                                      
CPU times: user 285 ms, sys: 58.4 ms, total: 343 ms
Wall time: 31.8 s


In [3]:
import pandas as pd
import numpy as np

# Analysis using Standard Pandas

Let's load the parquet dataset first-

In [4]:
%%time
df_transactions = pd.read_parquet('HI-Large_Trans_reduced.parquet', columns = ['Timestamp','From Bank','To Bank','Amount Received',
                                                                               'Payment Currency','Is Laundering'])
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89851114 entries, 0 to 89851113
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Timestamp         object 
 1   From Bank         int64  
 2   To Bank           int64  
 3   Amount Received   float64
 4   Payment Currency  object 
 5   Is Laundering     int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.0+ GB
CPU times: user 14.9 s, sys: 8.49 s, total: 23.4 s
Wall time: 16.3 s


Transaction table contains account details, transaction details, and a flag for laundering.

In [5]:
df_transactions.head()

,Timestamp,From Bank,To Bank,Amount Received,Payment Currency,Is Laundering
0,2022/08/01 00:02,3196,3196,7739.29,US Dollar,0
1,2022/08/01 00:03,1208,20,73966883.00,US Dollar,0
2,2022/08/01 00:27,3203,3203,13284.41,US Dollar,0
3,2022/08/01 00:09,1208,1208,7.66,US Dollar,0
4,2022/08/01 00:06,1208,1208,4.86,US Dollar,0


## Which banks have the most money laundering related transactions?

In [6]:
%%time
# Aggregate-
result=df_transactions.groupby(
    ["From Bank","To Bank","Payment Currency"]).agg({"Amount Received":"sum","Is Laundering":"sum"})

filtered_result = result[result["Is Laundering"] > 0].sort_values(by="Amount Received", ascending=False)
filtered_result.head(10)

CPU times: user 22.4 s, sys: 4.63 s, total: 27.1 s
Wall time: 27.2 s


,,,Amount Received,Is Laundering
From Bank,To Bank,Payment Currency,,
4011,4011,US Dollar,2.287563e+13,1
18824,18824,US Dollar,1.181472e+13,3
18184,4,Rupee,5.257959e+12,1
221118,221118,US Dollar,4.857862e+12,1
214853,214853,US Dollar,1.908466e+12,1
28781,28781,US Dollar,1.458829e+12,1
2310,2310,US Dollar,1.359950e+12,8
70,137768,Yen,1.216654e+12,9
5763,5763,US Dollar,1.087834e+12,2


It's interesting to see that most money laundering happen within (to and from) the same bank.

Also, 30 seconds is too long for a simple aggregation!

## Which locations are most highly correlated with money laundering related transaction?

It is helpful to understand the locations that have the most money laundering related activity to take appropriate steps.

For this, we will merge the location dataset with transaction dataset.

Let's load the location dataset first -

In [7]:
!gsutil cp gs://rapidsai/pno-oom-demo/account_locations.parquet /content/

Copying gs://rapidsai/pno-oom-demo/account_locations.parquet...
/ [1 files][  1.5 MiB/  1.5 MiB]                                                
Operation completed over 1 objects/1.5 MiB.                                      


In [8]:
%%time
df_location = pd.read_parquet('/content/account_locations.parquet')
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116329 entries, 0 to 116328
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  116329 non-null  int64 
 1   From Bank   116329 non-null  int64 
 2   location    116329 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.7+ MB
CPU times: user 35.9 ms, sys: 4.34 ms, total: 40.3 ms
Wall time: 29.8 ms


Merging the transaction data with the account locations on `From Bank` column which contains the account number.





In [9]:
%%time
df_merged = df_transactions.iloc[1:int(len(df_transactions)/2)].merge(
    df_location, how='left', on='From Bank')

CPU times: user 4.95 s, sys: 2.13 s, total: 7.08 s
Wall time: 7.08 s


Note we are using a different length of the transaactions dataset because the merge on the full dataset exhausts the full system memory and crashes the session.

We see that each operation is taking around 10 to 30 seconds to run with pandas.

In [10]:
%%time
# Aggregate-
result=df_merged.groupby(
    ["location"]).agg({"Amount Received":"sum","Is Laundering":"sum"})

filtered_result = result[result["Is Laundering"] > 0 ].sort_values(by="Amount Received", ascending=False)
filtered_result.head(10)

CPU times: user 3.8 s, sys: 666 ms, total: 4.46 s
Wall time: 4.49 s


,Amount Received,Is Laundering
location,,
San Antonio,4.877528e+13,5960
Los Angeles,4.397943e+13,5284
Dallas,4.297125e+13,5181
San Jose,3.172923e+13,4640
Philadelphia,2.596176e+13,10669
Chicago,1.926709e+13,4100
Phoenix,1.625138e+13,5107
San Diego,1.317739e+13,4131
Houston,1.184604e+13,3977


Note: This is based on mock data.

Banks in `San Antonio` and `Dallas` have the most transactions associated with money laundering.

# Analysis with `cudf.pandas`

In [6]:
# Restart notebook-
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

Let's first install cuDF's latest version-

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12==24.8.*

Typically, you should load the `cudf.pandas` extension as the first step in your notebook, before importing any modules. Here, we explicitly restart the kernel to simulate that behavior.

Note: We just added the `%load_ext` and the rest of the code remains the same.

In [2]:
%load_ext cudf.pandas

Before we load data, lets use the `rmm` library to make sure we are tracking GPU utilization. We just wrap a `StatisticsResourceAdaptor` on our memory resource to see what our memory allocations were for the upcoming operations.


In [3]:
import rmm
stats_mr = rmm.mr.StatisticsResourceAdaptor(
    rmm.mr.get_current_device_resource())
rmm.mr.set_current_device_resource(stats_mr)

 Let's use the rmm library to track GPU memory usage.

In [4]:
import pandas as pd

We'll run the same code as above to get a feel what GPU-acceleration brings to pandas workflows!

In [5]:
%%cudf.pandas.profile
df_transactions = pd.read_parquet('HI-Large_Trans_reduced.parquet')


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 97, in __enter__
    sys.settrace(self._tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 116, in __exit__
    sys.settrace(self._oldtrace)



                                                                                                  
                                    Total time elapsed: 3.278 seconds                             
                                  1 GPU function calls in 2.948 seconds                           
                                  0 CPU function calls in 0.000 seconds                           
                                                                                                  
                                                  Stats                                           
                                                                                                  
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function     ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ read_parquet │ 1          │ 2.948       │ 2.948       │ 0          │ 0.000       │ 0.000       │
└──────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [6]:
%%time
# Aggregate-
df_transactions[["From Bank","Payment Currency","Amount Received"]].groupby(
    ["From Bank","Payment Currency"]).agg({"Amount Received":"sum"})

CPU times: user 2.77 s, sys: 2.06 s, total: 4.83 s
Wall time: 6.05 s


Amount Received
From Bank Payment Currency                  
0         Australian Dollar     2.593470e+05
          Bitcoin               1.236489e+03
          Brazil Real           6.402262e+08
          Canadian Dollar       4.724067e+05
          Euro                  7.410547e+09
...                                      ...
3225441   Bitcoin               1.981200e-02
3225444   Bitcoin               4.904000e-02
3225451   US Dollar             2.820300e+02
3225454   US Dollar             3.459300e+03
3225455   Bitcoin               2.543745e+00

[144488 rows x 1 columns]

This was much faster than before! We were able to get the processing time down for `groupby` operation by 5x.


# Can we really handle workloads larger than GPU memory?

In [7]:
print(f"Total memory usage {round(stats_mr.allocation_counts.current_bytes/(1024**3),0)} GB")

Total memory usage 10.0 GB


In [8]:
print(f"Peak memory usage {round(stats_mr.allocation_counts.peak_bytes/(1024**3),0)} GB")

Peak memory usage 17.0 GB


It's interesting to see that peak memory usage was higher than the GPU memory (16 GB) and yet we saw speedups from 5 minutes to around 30 seconds. It is because of the ability to process larger than GPU memory workloads with `cudf.pandas`.


We are seeing speedups for large datasets with `cudf.pandas` without changing even a single line of pandas code! This can be attributed to better memory management attributed to managed memory pool and prefetching concepts we explained in the beginning of the notebook. see this <a href="https://developer.nvidia.com/blog/improving-gpu-memory-oversubscription-performance/">NVIDIA Developer blog post</a> for more details.

## But what happens if we switch off managed memory feature?

In [9]:
# Restart notebook-
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

`cudf.pandas` provides an environment variable `CUDF_PANDAS_RMM_MODE` that you can set to `cuda` to turn off managed memory. For more details see, https://docs.rapids.ai/api/cudf/stable/cudf_pandas/how-it-works/ .

In [ ]:
%env CUDF_PANDAS_RMM_MODE=cuda

import os
# Step 3: Verify the environment variable
print(os.environ['CUDF_PANDAS_RMM_MODE'])

In [2]:
%load_ext cudf.pandas
import pandas as pd

**WARNING**: Executing the below cell would crash your colab session due to system memory constraints -- managed memory prevented that in the previous run

In [ ]:
%%cudf.pandas.profile
df_transactions = pd.read_parquet('/content/HI-Large_Trans_reduced.parquet')

Note: The above cell would crash your colab session on a free tier because

1. Without managed memory GPU couldn't load the entire dataset, leading to `out of memory` issues
2. The execution falls back to CPU gracefully
3. CPU memory limit of 12.7 GB (on free tier) couldn't handle a 10 GB dataset because of higher peak memory requirements.

This didn't happen in earlier execution of `cudf.pandas` with managed memory (set by default). Seamless execution between CPU and GPU through managed memory comes to resue!

# Summary

With cudf.pandas, you can keep using pandas as your primary dataframe library. When things start to get a little slow, just load the `cudf.pandas` extension and enjoy the incredible speedups.

If you like Google Colab and want to get peak `cudf.pandas` performance to process even larger datasets, Google Colab's paid tier includes both L4 and A100 GPUs (in addition to the T4 GPU this demo notebook is using).

To learn more about cudf.pandas, we encourage you to visit https://rapids.ai/cudf-pandas.

# Do you have any feedback for us?

Fill this quick survey <a href="https://www.surveymonkey.com/r/TX3QQQR">HERE</a>

Raise an issue on our github repo <a href="https://github.com/rapidsai/cudf/issues">HERE</a>